# Combinatorial Analysis of Baccarat


[Rules of Baccarat (Punto Banco)](https://en.wikipedia.org/wiki/Baccarat_(card_game)#Punto_banco)

Inspired by a [method described](https://www.888casino.com/blog/baccarat-tips/baccarat-combinatorial-analysis-the-easy-way) by Eliot Jacobson.

> What makes CA easy for baccarat is the simple observation that every hand in baccarat is completely determined by a sequence of six cards. While it is true that the 5th and 6th cards may not be used, by always including them as a possibility, the baccarat universe can be fully written out [...]

The general strategy is to count how many possible 6 card hands result in a given outcome (banker, tie, player) and divide that number by the total amount of possible 6 card hands to get a probability.

Consider all possible hands would be a bit slow so to speed up this computation, let's first just consider all "unique" hands (e.g. we assume that cards 10, J, Q, K are all equivalent).

In [2]:
import itertools

# Since cards 10, J, Q, K are all worth 10 so we just include 10 to avoid counting equivalent hands.
# For example, (10, 2, 3, 4, 5, 6) is the same as (J, 2, 3, 4, 5, 6).
card_values = [1,2,3,4,5,6,7,8,9,10]

# Enumerate all 6 card unique hands
unique_hands = list(itertools.product(card_values, repeat=6))

len(unique_hands)

1000000

For each unique hand, we will need to know:

1. Its outcome (banker, player, tie).
2. The number of non unique ways to draw it from a shoe (a shoe is typically 8 decks of cards). For example, the unique hand `(10, 2, 3, 4, 5, 6)` can be drawn in 4 different ways: `(10, 2, 3, 4, 5, 6)`, `(J, 2, 3, 4, 5, 6)`, `(Q, 2, 3, 4, 5, 6)`, `(K, 2, 3, 4, 5, 6)`.

Knowing this, we can count how many possible hands lead to each outcome and divide that number by the total number of possible hands to get a probability for each outcome. 

In [4]:
n_suites = 4
def count_hands(unique_hand, n_decks = 8):
  card_count = n_suites * n_decks
  # For each value, how many actual card produce it.
  card_counts = {
      '1': card_count,
      '2': card_count,
      '3': card_count,
      '4': card_count,
      '5': card_count,
      '6': card_count,
      '7': card_count,
      '8': card_count,
      '9': card_count,
      '10': card_count * len(['10', 'J', 'Q', 'K'])
  }
  # count possible ways to form hand from shoe
  count = 1
  for c in unique_hand:
    count = count * card_counts[str(c)]
    # "remove card from shoe"
    card_counts[str(c)] -= 1
  return count

example_hand = (5, 1, 3, 2, 2, 6)
count_hands(example_hand)

1040187392

Now, for each of those unique hands, we will need to count how many result in a tie, banker win or player win. Let's write a function that takes a 6 card hand and computes the outcome:

In [5]:
# In Baccarat, when cards are added and the result is over 10, and only the last digit is kept.
def bacc_add(n1, n2):
  return (n1 + n2) % 10

# Computing the outcome of a hand according to the somewhat complex rules of Baccarat. 
def compute_outcome(hand):
  [c1,c2,c3,c4,c5,c6] = hand
  banker_total = bacc_add(c1, c2)
  player_total = bacc_add(c3, c4)
  
  if (player_total == 6 or player_total == 7):
    if (banker_total <= 5):
      banker_total = bacc_add(banker_total, c5)
  elif (player_total <= 5 and banker_total < 8):
    player_total = bacc_add(player_total, c5)
    if (banker_total <= 2):
      banker_total = bacc_add(banker_total, c6)
    elif (banker_total == 3 and c5 != 8):
      banker_total = bacc_add(banker_total, c6)
    elif (banker_total == 4 and c5 in [2, 3, 4, 5, 6, 7]):
      banker_total = bacc_add(banker_total, c6)
    elif (banker_total == 5 and c5 in [4, 5, 6, 7]):
      banker_total = bacc_add(banker_total, c6)
    elif (banker_total == 6 and c5 in [6, 7]):
      banker_total = bacc_add(banker_total, c6)
  
  if (banker_total > player_total):
    return 'banker'
  elif (player_total > banker_total):
    return 'player'
  return 'tie'
  
example_hand = (5, 1, 3, 2, 2, 6)
compute_outcome(example_hand)

'player'

We are ready to compute the outcome and hand count of each unique hand.

In [6]:
counts = { 'tie':0, 'banker':0, 'player':0 }
for hand in unique_hands:
  outcome = compute_outcome(hand)
  count = count_hands(hand)
  counts[outcome] += count

print(counts)

{'tie': 475627426473216, 'banker': 2292252566437888, 'player': 2230518282592256}


We can now compute the probability for each outcome by diviging the number of hands for each by the number of possible hands.

In [11]:
tie = counts['tie']
banker = counts['banker']
player = counts['player']
total_count = tie + banker + player


print('Assuming an 8 decks shoe:')
print('')
print('total count', total_count)

print('probabilities:')
print('tie:', tie / total_count)
print('banker:', banker / total_count)
print('player:', player / total_count)

Assuming an 8 decks shoe:

total count 4998398275503360
probabilities:
tie: 0.0951559680236402
banker: 0.458597422632763
player: 0.44624660934359683
